In [1]:
#デフォルト回路の読み込み
from pynq import Overlay
# load Base Overlay
Overlay("/home/xilinx/pynq/bitstream/base.bit").download()

Pmodに接続されているIPをPythonで操作する。方法はいくつかある。Pmod_LED8クラスのような専用クラスもある。

In [2]:
from pynq.iop import Pmod_LED8
leds = Pmod_LED8(1,1)
leds.on()
leds.write(0)
leds.toggle()

今回はPmod_IOクラスを使う。Pmodの任意のピン(A/B,0-7)の入出力設定と出力を操作できる。
以下はサンプルスクリプト。

In [3]:
from pynq.iop import Pmod_IO
sample = Pmod_IO(1, 1, 'out')
sample.write(1)

In [4]:
#pinsリストにPmod_IOクラスオブジェクトを入れる。リストの要素のwriteメソッドを呼び出せばピンの挙動を操作できる。
from pynq.iop import Pmod_IO
pins = []
for i in range(8):
    pins.append(Pmod_IO(1, i, 'out'))
state = [0,0,0,0,0,0,1,0]
for i in range(8):
    pins[i].write(state[i])

サンプルプログラム  
数を2進数にしてから、ビットシフト。bin(i)ではstr型なのでint(,base=2)で2進数intに変換してから、「かつ」を行う。

In [10]:
b = int(bin(5>>3),base=2)
b & 0b0000001

0

In [6]:
#Pmodに0b00000000から0b11111111まですべてを代入する
from pynq.iop import Pmod_IO
import time
pins = []
for i in range(8):
    pins.append(Pmod_IO(1, i, 'out'))
    
for i in range(256):
    for j in range(8):
        pins[j].write(int(bin(i>>j),base=2) & 0b00000001)
    print(bin(i))
    time.sleep(.1)      

0b0
0b1
0b10
0b11
0b100
0b101
0b110
0b111
0b1000
0b1001
0b1010
0b1011
0b1100
0b1101
0b1110
0b1111
0b10000
0b10001
0b10010
0b10011
0b10100
0b10101
0b10110
0b10111
0b11000
0b11001
0b11010
0b11011
0b11100
0b11101
0b11110
0b11111
0b100000
0b100001
0b100010
0b100011
0b100100
0b100101
0b100110
0b100111
0b101000
0b101001
0b101010
0b101011
0b101100
0b101101
0b101110
0b101111
0b110000
0b110001
0b110010
0b110011
0b110100
0b110101
0b110110
0b110111
0b111000
0b111001
0b111010
0b111011
0b111100
0b111101
0b111110
0b111111
0b1000000
0b1000001
0b1000010
0b1000011
0b1000100
0b1000101
0b1000110
0b1000111
0b1001000
0b1001001
0b1001010
0b1001011
0b1001100
0b1001101
0b1001110
0b1001111
0b1010000
0b1010001
0b1010010
0b1010011
0b1010100
0b1010101
0b1010110
0b1010111
0b1011000
0b1011001
0b1011010
0b1011011
0b1011100
0b1011101
0b1011110
0b1011111
0b1100000
0b1100001
0b1100010
0b1100011
0b1100100
0b1100101
0b1100110
0b1100111
0b1101000
0b1101001
0b1101010
0b1101011
0b1101100
0b1101101
0b1101110
0b1101111
0b1110

pin7がHの時は左サイド(ブザー側)の7セグが光る。Lのときは右サイドの7セグが光る。

このPmodモジュールの上にあるのトランジスタはNPN型。文字がある側の左から、エミッタ(E)、コレクタ(C)、ベース(B)である。Bに電流が流れると、EとCの間が通電する。電気スイッチの役割を果たしている。

In [7]:
#ブザーを鳴らす。pin7のH/Lを繰り返す。sleepが短いほど高い音になる。
#SW0がHになっている間はなり続けるが、Lになると止まりendと表示して終わる。
from pynq.iop import Pmod_IO
import time
from pynq.board import Switch
pin = Pmod_IO(1, 7, 'out')
switch = Switch(0)
while switch.read():
    pin.write(1)
    time.sleep(.002)
    pin.write(0)
    
print('end')

end


7セグを操作するクラスを用意する。

In [8]:
from pynq.iop import Pmod_IO
#ピンの設定
pins = []
for i in range(8):
    pins.append(Pmod_IO(1, i, 'out'))
class Seg7(object):
    out = [[]*8]*6
    out[0] = [1,1,1,1,0,1,1,0]
    out[1] = [0,1,0,0,0,0,1,0]
    out[2] = [0,1,1,1,1,1,0,0]
    out[3] = [0,1,1,0,1,1,1,0]
    out[4] = [1,1,0,0,1,0,1,0]
    out[5] = [1,0,1,0,1,1,1,0]
    
    def output(self, num):
        for i in range(8):
          pins[i].write(self.out[num][i])
    def clear(self):
        for i in range(8):
          pins[i].write(0)        

5から0へカウントダウンする。

In [9]:
import time
seg = Seg7()
for i in range(6):
    seg.output(5-i)
    time.sleep(1)
    seg.clear()
    